# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd
import numpy as np 

In [3]:
# Your code goes here
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv'
data = pd.read_csv(url)
marketing_df = data.copy()
pd.set_option('display.max_columns', None)
print(marketing_df.shape)
marketing_df.head()

(10910, 27)


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,Suburban,Married,61,7.000000,52,0.000000,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,Suburban,Single,64,3.000000,26,0.000000,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,Suburban,Single,100,34.000000,31,0.000000,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,Suburban,Single,97,10.000000,3,0.000000,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,Suburban,Married,117,15.149071,31,0.384256,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [4]:
total_claim_marketing_df = marketing_df[(marketing_df['total_claim_amount'] > 1000.0) & (marketing_df['response'] == 'Yes')]
total_claim_marketing_df

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,Suburban,Single,283,16.0,43,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A,1
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,Suburban,Single,283,16.0,43,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A,1
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2011-02-13,Employed,F,71210,Suburban,Married,214,8.0,89,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A,2
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2011-02-13,Disabled,F,16181,Suburban,Divorced,181,22.0,79,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A,2
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,2011-01-26,Unemployed,M,0,Suburban,Married,153,25.0,5,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,10351,FN44127,Oregon,3508.569533,Yes,Extended,College,2011-01-05,Medical Leave,M,20978,Suburban,Married,98,19.0,96,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,A,1
10373,10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2011-02-08,Employed,M,55687,Suburban,Divorced,276,33.0,91,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,A,2
10487,10487,IX60941,Oregon,3508.569533,Yes,Extended,College,2011-01-05,Medical Leave,M,20978,Suburban,Married,98,19.0,96,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,A,1
10565,10565,QO62792,Oregon,7840.165778,Yes,Extended,College,2011-01-14,Employed,M,58414,Suburban,Single,210,7.0,24,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,Four-Door Car,Medsize,A,1


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [5]:
# Creating a new dataframe filtering the customers who have responded "yes" to the las marketing campaign
marketing_agg_df = marketing_df[marketing_df['response']== 'Yes']

# Average of total_claim_amount by policy type and gender
marketing_agg_df = marketing_agg_df.groupby(["policy_type", "gender"])["total_claim_amount"].agg(["mean"]) 
marketing_agg_df

mean
policy_type    gender            
Corporate Auto F       433.738499
               M       408.582459
Personal Auto  F       452.965929
               M       457.010178
Special Auto   F       453.280164
               M       429.527942

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [13]:
# Total number of cusotobers by state and policy type
cust_agg_df = marketing_df.groupby(["state", "policy_type"])["customer"].agg(["count"]).round(2) 
cust_agg_df

count
state      policy_type          
Arizona    Corporate Auto    381
           Personal Auto    1469
           Special Auto       87
California Corporate Auto    974
           Personal Auto    3058
           Special Auto      151
Nevada     Corporate Auto    219
           Personal Auto     739
           Special Auto       35
Oregon     Corporate Auto    592
           Personal Auto    2180
           Special Auto      137
Washington Corporate Auto    175
           Personal Auto     682
           Special Auto       31

In [14]:
cust_agg_df.columns

Index(['count'], dtype='object')

In [15]:
# Filter states with more than 500 customers
cust_agg_df = cust_agg_df[cust_agg_df['count']>500]
cust_agg_df

count
state      policy_type          
Arizona    Personal Auto    1469
California Corporate Auto    974
           Personal Auto    3058
Nevada     Personal Auto     739
Oregon     Corporate Auto    592
           Personal Auto    2180
Washington Personal Auto     682

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [10]:
# Average, max and min of customer_lifetime_value by education level and gender
cust_agg_df = marketing_df.groupby(["education", "gender"])["customer_lifetime_value"].agg(["max", "min","mean"]).round(2) 
cust_agg_df

max      min     mean
education            gender                            
Bachelor             F       73225.96  1904.00  7874.27
                     M       67907.27  1898.01  7703.60
College              F       61850.19  1898.68  7748.82
                     M       61134.68  1918.12  8052.46
Doctor               F       44856.11  2395.57  7328.51
                     M       32677.34  2267.60  7415.33
High School or Below F       55277.45  2144.92  8675.22
                     M       83325.38  1940.98  8149.69
Master               F       51016.07  2417.78  8157.05
                     M       50568.26  2272.31  8168.83

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [16]:
# your code goes here
# Number of policies sold by state and month
st_month_df = marketing_df.groupby(["state", "month"])["policy_type"].agg(["count"]).round(2) 
st_month_df

count
state      month       
Arizona    1       1008
           2        929
California 1       2231
           2       1952
Nevada     1        551
           2        442
Oregon     1       1565
           2       1344
Washington 1        463
           2        425

In [17]:
# Pivot the DataFrame to see the month as column and state aas rows
st_month_df = st_month_df.pivot_table(index='state', columns='month', values=['count'])
st_month_df

count        
month            1       2
state                     
Arizona     1008.0   929.0
California  2231.0  1952.0
Nevada       551.0   442.0
Oregon      1565.0  1344.0
Washington   463.0   425.0

In [19]:
st_month_df.columns

MultiIndex([('count', 1),
            ('count', 2)],
           names=[None, 'month'])

In [25]:
# number of policies sold by month and state
# Sort values by count column in descending order
st_month_df = st_month_df.sort_values(by = [('count', 1),('count', 2)], ascending= False)
st_month_df

count        
month            1       2
state                     
California  2231.0  1952.0
Oregon      1565.0  1344.0
Arizona     1008.0   929.0
Nevada       551.0   442.0
Washington   463.0   425.0

In [29]:
# Top 3 states with the highest number of policies sold
top_3_dt = st_month_df.iloc[0:3]
top_3_dt

count        
month            1       2
state                     
California  2231.0  1952.0
Oregon      1565.0  1344.0
Arizona     1008.0   929.0

In [34]:
# Effect of different marketing channels on the customer response rate.
# Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.
chan_res_df = marketing_df.groupby(["sales_channel","response"])["response"].agg(["count"]).round(2) 
chan_res_df


count
sales_channel response       
Agent         No         3379
              Yes         742
Branch        No         2696
              Yes         326
Call Center   No         1920
              Yes         221
Web           No         1449
              Yes         177

In [41]:
# Reset index
chan_res_df = chan_res_df.reset_index()

RangeIndex(start=0, stop=8, step=1)

In [43]:
chan_res_pivot_df = chan_res_df.pivot(index='sales_channel', columns='response', values='count')
chan_res_pivot_df

response,No,Yes
sales_channel,,
Agent,3379,742
Branch,2696,326
Call Center,1920,221
Web,1449,177


In [44]:
chan_res_pivot_df.columns

Index(['No', 'Yes'], dtype='object', name='response')

In [45]:
chan_res_pivot_df['customer_response_rate'] = chan_res_pivot_df['Yes'] / (chan_res_pivot_df['No'] +chan_res_pivot_df['Yes'] )
chan_res_pivot_df

response,No,Yes,customer_response_rate
sales_channel,,,
Agent,3379,742,0.180053
Branch,2696,326,0.107876
Call Center,1920,221,0.103223
Web,1449,177,0.108856
